In [1]:
pip install datasets faiss-gpu sentence_transformers gradio rank_bm25 openai==0.28


In [2]:
from datasets import load_dataset
import gradio as gr
import pandas as pd
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import nltk
from nltk.tokenize import word_tokenize
import openai
import torch
import re


In [3]:

ds = load_dataset("traversaal-ai-hackathon/hotel_datasets")

df = ds['train'].to_pandas()

csv_path = 'hotel_datasets.csv'
df.to_csv(csv_path, index=False)

print(f"The dataset has been saved to {csv_path}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The dataset has been saved to hotel_datasets.csv


In [4]:
print(df.shape)
print(df.locality.unique())
df.head()

(5997, 14)
['Istanbul' 'San Francisco' 'London' 'New York City' 'Paris']


,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",None,NaN,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,None,NaN,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",None,NaN,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,None,NaN,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye


In [5]:

df = pd.read_csv('hotel_datasets.csv')
df['review_text']=df['review_text'].fillna('there is no review text')
df['rate']=df['rate'].fillna('there is no rate')
df['combined'] = ' hotel description ' + df['hotel_description'] + ' review title ' + df['review_title'] + ' review text ' + df['review_text']


In [6]:
df.columns

Index(['hotel_name', 'hotel_description', 'review_title', 'review_text',
       'rate', 'tripdate', 'hotel_url', 'hotel_image', 'price_range',
       'rating_value', 'review_count', 'street_address', 'locality', 'country',
       'combined'],
      dtype='object')

In [7]:
df.combined[0]



' hotel description Romance Istanbul Hotel has 39 rooms.Every room is elegantly furnished and harmonizes the modern life style with the traditional Ottoman touch. Romance Istanbul sits at the intersection of the old city’s most important part. With its luxuriously inspiring design and landmark old city location, steeped in the history of its surroundings, Romance Istanbul Hotel welcomes you with exceptional designed rooms and world-renowned Turkish hospitality. Our colleagues deliver the most personal service. It is perfectly placed and perfectly designed to enhance all that Istanbul has to offer. Each room offers a private bathroom and shower. Each is equipped with a satellite TV and free wifi connection. The rooms size change between 20 m2 and 45 m2. It includes 7 suite rooms: 1 Royal Suite, 4 Grand Suite, 1 Romance Suite and 1 Premium Suite, 2 Luxury Room With Terrace, 22 Deluxe Room, 8 City Room. review title An exceptional boutique hotel, great value for your money review text the

In [8]:

df_combined = df.copy()

df_combined['combined'] = df_combined['combined'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',str(x)))

def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_combined['combined']= df_combined['combined'].apply(lambda x: lower_case(x))

In [9]:

embedder = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)
embedder.to('cuda')
df_combined["embedding"] = df_combined.combined.apply(lambda x: embedder.encode(x))
df.head()


,hotel_name,hotel_description,review_title,review_text,rate,tripdate,hotel_url,hotel_image,price_range,rating_value,review_count,street_address,locality,country,combined
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"An exceptional boutique hotel, great value for...",there is no review text,there is no rate,February 2020,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,hotel description Romance Istanbul Hotel has ...
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,You can’t get better than this.,there is no review text,there is no rate,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,hotel description Romance Istanbul Hotel has ...
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Exceeds all expectations,there is no review text,there is no rate,March 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,hotel description Romance Istanbul Hotel has ...
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,"Great Location, Fantastic Accommodations",there is no review text,there is no rate,August 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,hotel description Romance Istanbul Hotel has ...
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Perfection. It is all in the details.,there is no review text,there is no rate,June 2021,https://www.tripadvisor.com/Hotel_Review-g2939...,https://media-cdn.tripadvisor.com/media/photo-...,$ (Based on Average Nightly Rates for a Standa...,5.0,4023,Hudavendigar Cd. No:5 Sirkeci,Istanbul,Turkiye,hotel description Romance Istanbul Hotel has ...


In [10]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
df_combined.to_pickle('/content/drive/MyDrive/Semantic_Search/entire_data.pkl')

In [33]:
kay = pd.read_json('/content/j.json', orient='index')


In [40]:
key=kay.values[0][0]

In [41]:


df = pd.read_pickle('/content/drive/MyDrive/Semantic_Search/entire_data.pkl')

embedder = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)
embedder.to('cuda')
embeddings = np.stack(df['embedding'].values)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

openai.api_key = key
nltk.download('punkt')

def search_reviews_faiss(query, n=5):
    query_embedding = embedder.encode(query)
    distances, indices = index.search(query_embedding.reshape(1, -1), n)
    results = df.iloc[indices[0]]
    return results

def get_chatgpt_response(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "do summarization for the hotels after that give a recommendation in conclusion of what is the best based on the query and the url of the hotel, do not write it as markdown"},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        n=1,
        stop=None,
        temperature=0.8,
    )
    return response.choices[0].message['content'].strip()

def process_query(query):
    faiss_results = search_reviews_faiss(query, n=10)
    faiss_results['tokenized'] = faiss_results['combined'].apply(lambda x: word_tokenize(x.lower()))

    bm25 = BM25Okapi(faiss_results['tokenized'].tolist())
    tokenized_query = word_tokenize(query.lower())
    bm25_scores = bm25.get_scores(tokenized_query)
    faiss_results['bm25_score'] = bm25_scores
    faiss_results = faiss_results.sort_values(by='bm25_score', ascending=False)
    top_results = faiss_results.head(5)

    top_results['all'] = 'Hotel name '+top_results['hotel_name']+' Rate '+top_results['rate'].astype(str)+' hotel url '+top_results['hotel_url']+' review count '+top_results['review_count'].astype(str)+' locality '+top_results['locality']+' country '+top_results['country']+' hotel description ' + top_results['combined']
    s = ''
    for i in top_results['all']:
        s += i + '\n\n\n'

    prompt = f"the query {query}\n hotels: {s}"
    response = get_chatgpt_response(prompt)
    return response

iface = gr.Interface(
    fn=process_query,
    inputs="text",
    outputs="text",
    title="Hotel Search and Recommendation",
    description="Enter a query to search for hotels and get recommendations."
)

if __name__ == "__main__":
    iface.launch()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://afaa7bf173dd4e92ab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
